In [1]:
import torch 
import json,pickle,math
import pandas as pd
import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


In [2]:
LSM = pickle.load(open('./davis_ligand_similarity_matrix.pkl', 'rb'))
PSM = pickle.load(open('./davis_protein_similarity_matrix.pkl', 'rb'))
df = pd.read_csv(open('./all_pairs.csv','r'))

In [3]:
# df

In [4]:
SMILES = json.load(open('./data/DAVIS/SMILES.txt'))
TARGETS = json.load(open('./data/DAVIS/target_seq.txt'))
SMILES=list(SMILES.values())
TARGETS=list(TARGETS.values())

In [5]:
outer_prods = []
for i,row in df.iterrows():
#     print(i)
    smi = row['SMILES']
    seq = row['Target Sequence']
    target_id = TARGETS.index(seq)
    smi_id = SMILES.index(smi)
    ki=LSM[smi_id]
    kj=PSM[target_id]
    ki_x_kj = np.outer(ki,kj)
    outer_prods.append([ki_x_kj])
outer_prods = np.array(outer_prods)
print(np.shape(outer_prods))

(30056, 1, 68, 442)


In [6]:
class custom_dataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, outer_prods, transform=None):
        self.df = pd.read_csv(open(csv_file))
#         self.root_dir = root_dir
        self.transform = transform
        self.outer_prods = outer_prods
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        output = {'outer_product': self.outer_prods[idx] , 'Label':self.df.iloc[idx]['Label']}
        return output

In [7]:
dataset = custom_dataset(csv_file = './all_pairs.csv', outer_prods = outer_prods)
full_dataset = dataset

In [8]:
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

In [9]:
train_loader= torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader= torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

In [10]:
# Hyper parameters
num_epochs = 5
# num_classes = 10
batch_size = 32
learning_rate = 0.001

In [11]:
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1,32, 5).double()
        self.pool1 = nn.MaxPool2d(2,2).double()
        self.conv2 = nn.Conv2d(32,18,3).double()
        self.pool2 = nn.MaxPool2d(2,2).double()
        self.fc1 = nn.Linear(18*15*108, 128).double()
        self.fc2 = nn.Linear(128,1).double()
        self.dropout = nn.Dropout(0.1).double()
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1,18*15*108)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        
        return x
    

In [12]:
# for i in test_loader:
#     a = i['outer_product']
#     b= i['Label']
#     break
# conv1 = nn.Conv2d(1,32,5).double()
# pool = nn.MaxPool2d(2,2).double()
# conv2 = nn.Conv2d(32,18,3).double()
# fc1 = nn.Linear(18*15*108, 128).double()
# fc2 = nn.Linear(128,1).double()
# dropout = nn.Dropout(0.1).double()
# x= conv1(a)
# x.shape
# x = pool(x)
# x.shape
# x= conv2(x)
# x.shape
# x = pool(x)
# x.shape
# x = x.view(-1,18*15*108)
# x.shape
# x = dropout(x)
# x.shape
# x = fc1(x)
# x.shape
# x = fc2(x)
# x.shape

In [13]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 20
# num_classes = 10
batch_size = 32
learning_rate = 0.001
model = ConvNet().to(device)


In [14]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [15]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    c=0
    for i in train_loader:
        c=c+1
        images = i['outer_product']
        labels = i['Label']
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
     
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
               .format(epoch+1, num_epochs, c, total_step, loss.item()))


/home2/kanakala.ganesh/miniconda3/envs/geometric/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448238472/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/home2/kanakala.ganesh/miniconda3/envs/geometric/lib/python3.9/site-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/20], Step [1/752], Loss: 30.0207
Epoch [1/20], Step [2/752], Loss: 9.8105
Epoch [1/20], Step [3/752], Loss: 1.5411
Epoch [1/20], Step [4/752], Loss: 11.9239
Epoch [1/20], Step [5/752], Loss: 4.8648
Epoch [1/20], Step [6/752], Loss: 1.0024
Epoch [1/20], Step [7/752], Loss: 1.5601
Epoch [1/20], Step [8/752], Loss: 4.9392
Epoch [1/20], Step [9/752], Loss: 3.7360
Epoch [1/20], Step [10/752], Loss: 2.7668
Epoch [1/20], Step [11/752], Loss: 1.2026
Epoch [1/20], Step [12/752], Loss: 0.7114
Epoch [1/20], Step [13/752], Loss: 1.0702
Epoch [1/20], Step [14/752], Loss: 1.7618
Epoch [1/20], Step [15/752], Loss: 1.7337
Epoch [1/20], Step [16/752], Loss: 2.1623
Epoch [1/20], Step [17/752], Loss: 0.9807
Epoch [1/20], Step [18/752], Loss: 0.5986
Epoch [1/20], Step [19/752], Loss: 1.0402
Epoch [1/20], Step [20/752], Loss: 1.8882
Epoch [1/20], Step [21/752], Loss: 1.0489
Epoch [1/20], Step [22/752], Loss: 1.9733
Epoch [1/20], Step [23/752], Loss: 1.2115
Epoch [1/20], Step [24/752], Loss: 0.8504

Epoch [1/20], Step [196/752], Loss: 0.8470
Epoch [1/20], Step [197/752], Loss: 1.4405
Epoch [1/20], Step [198/752], Loss: 1.1547
Epoch [1/20], Step [199/752], Loss: 1.4178
Epoch [1/20], Step [200/752], Loss: 0.5528
Epoch [1/20], Step [201/752], Loss: 1.0850
Epoch [1/20], Step [202/752], Loss: 0.4910
Epoch [1/20], Step [203/752], Loss: 0.5551
Epoch [1/20], Step [204/752], Loss: 0.9144
Epoch [1/20], Step [205/752], Loss: 1.9307
Epoch [1/20], Step [206/752], Loss: 1.0758
Epoch [1/20], Step [207/752], Loss: 0.2213
Epoch [1/20], Step [208/752], Loss: 0.6334
Epoch [1/20], Step [209/752], Loss: 1.2713
Epoch [1/20], Step [210/752], Loss: 0.6294
Epoch [1/20], Step [211/752], Loss: 1.3584
Epoch [1/20], Step [212/752], Loss: 0.7884
Epoch [1/20], Step [213/752], Loss: 0.4873
Epoch [1/20], Step [214/752], Loss: 0.8898
Epoch [1/20], Step [215/752], Loss: 0.2842
Epoch [1/20], Step [216/752], Loss: 1.3358
Epoch [1/20], Step [217/752], Loss: 0.1960
Epoch [1/20], Step [218/752], Loss: 1.0215
Epoch [1/20

Epoch [1/20], Step [388/752], Loss: 0.9875
Epoch [1/20], Step [389/752], Loss: 1.0936
Epoch [1/20], Step [390/752], Loss: 0.4662
Epoch [1/20], Step [391/752], Loss: 0.3851
Epoch [1/20], Step [392/752], Loss: 0.4334
Epoch [1/20], Step [393/752], Loss: 0.6354
Epoch [1/20], Step [394/752], Loss: 0.4257
Epoch [1/20], Step [395/752], Loss: 1.4437
Epoch [1/20], Step [396/752], Loss: 0.8581
Epoch [1/20], Step [397/752], Loss: 0.2800
Epoch [1/20], Step [398/752], Loss: 0.8464
Epoch [1/20], Step [399/752], Loss: 0.3818
Epoch [1/20], Step [400/752], Loss: 0.2446
Epoch [1/20], Step [401/752], Loss: 0.9194
Epoch [1/20], Step [402/752], Loss: 0.7404
Epoch [1/20], Step [403/752], Loss: 1.1008
Epoch [1/20], Step [404/752], Loss: 1.4339
Epoch [1/20], Step [405/752], Loss: 1.1311
Epoch [1/20], Step [406/752], Loss: 1.0091
Epoch [1/20], Step [407/752], Loss: 1.1209
Epoch [1/20], Step [408/752], Loss: 0.4914
Epoch [1/20], Step [409/752], Loss: 0.4278
Epoch [1/20], Step [410/752], Loss: 0.4299
Epoch [1/20

Epoch [1/20], Step [580/752], Loss: 0.7473
Epoch [1/20], Step [581/752], Loss: 0.8905
Epoch [1/20], Step [582/752], Loss: 0.7082
Epoch [1/20], Step [583/752], Loss: 0.7877
Epoch [1/20], Step [584/752], Loss: 1.1154
Epoch [1/20], Step [585/752], Loss: 0.8262
Epoch [1/20], Step [586/752], Loss: 1.0820
Epoch [1/20], Step [587/752], Loss: 0.4891
Epoch [1/20], Step [588/752], Loss: 0.5852
Epoch [1/20], Step [589/752], Loss: 0.3272
Epoch [1/20], Step [590/752], Loss: 0.7815
Epoch [1/20], Step [591/752], Loss: 0.7442
Epoch [1/20], Step [592/752], Loss: 0.4544
Epoch [1/20], Step [593/752], Loss: 1.0413
Epoch [1/20], Step [594/752], Loss: 0.6636
Epoch [1/20], Step [595/752], Loss: 0.9736
Epoch [1/20], Step [596/752], Loss: 0.9927
Epoch [1/20], Step [597/752], Loss: 1.1758
Epoch [1/20], Step [598/752], Loss: 0.7266
Epoch [1/20], Step [599/752], Loss: 0.2440
Epoch [1/20], Step [600/752], Loss: 0.4014
Epoch [1/20], Step [601/752], Loss: 1.1318
Epoch [1/20], Step [602/752], Loss: 0.6148
Epoch [1/20

...

Epoch [19/20], Step [568/752], Loss: 0.7017
Epoch [19/20], Step [569/752], Loss: 0.3496
Epoch [19/20], Step [570/752], Loss: 1.0360
Epoch [19/20], Step [571/752], Loss: 0.4202
Epoch [19/20], Step [572/752], Loss: 0.2216
Epoch [19/20], Step [573/752], Loss: 0.2095
Epoch [19/20], Step [574/752], Loss: 1.1606
Epoch [19/20], Step [575/752], Loss: 0.5806
Epoch [19/20], Step [576/752], Loss: 0.1421
Epoch [19/20], Step [577/752], Loss: 1.0151
Epoch [19/20], Step [578/752], Loss: 0.7331
Epoch [19/20], Step [579/752], Loss: 1.0280
Epoch [19/20], Step [580/752], Loss: 0.3711
Epoch [19/20], Step [581/752], Loss: 0.7553
Epoch [19/20], Step [582/752], Loss: 0.5614
Epoch [19/20], Step [583/752], Loss: 0.3384
Epoch [19/20], Step [584/752], Loss: 1.0873
Epoch [19/20], Step [585/752], Loss: 0.8301
Epoch [19/20], Step [586/752], Loss: 0.9851
Epoch [19/20], Step [587/752], Loss: 0.7601
Epoch [19/20], Step [588/752], Loss: 1.6364
Epoch [19/20], Step [589/752], Loss: 0.8104
Epoch [19/20], Step [590/752], L

Epoch [20/20], Step [3/752], Loss: 0.4072
Epoch [20/20], Step [4/752], Loss: 0.4888
Epoch [20/20], Step [5/752], Loss: 1.0613
Epoch [20/20], Step [6/752], Loss: 0.6136
Epoch [20/20], Step [7/752], Loss: 0.5181
Epoch [20/20], Step [8/752], Loss: 0.4270
Epoch [20/20], Step [9/752], Loss: 0.5123
Epoch [20/20], Step [10/752], Loss: 0.9250
Epoch [20/20], Step [11/752], Loss: 0.5989
Epoch [20/20], Step [12/752], Loss: 1.0642
Epoch [20/20], Step [13/752], Loss: 0.3534
Epoch [20/20], Step [14/752], Loss: 0.3579
Epoch [20/20], Step [15/752], Loss: 0.7259
Epoch [20/20], Step [16/752], Loss: 0.5848
Epoch [20/20], Step [17/752], Loss: 0.3380
Epoch [20/20], Step [18/752], Loss: 0.3132
Epoch [20/20], Step [19/752], Loss: 0.9862
Epoch [20/20], Step [20/752], Loss: 1.2126
Epoch [20/20], Step [21/752], Loss: 0.8688
Epoch [20/20], Step [22/752], Loss: 0.5037
Epoch [20/20], Step [23/752], Loss: 0.8828
Epoch [20/20], Step [24/752], Loss: 1.2523
Epoch [20/20], Step [25/752], Loss: 0.4614
Epoch [20/20], Ste

Epoch [20/20], Step [192/752], Loss: 1.3169
Epoch [20/20], Step [193/752], Loss: 1.9794
Epoch [20/20], Step [194/752], Loss: 1.3751
Epoch [20/20], Step [195/752], Loss: 0.6791
Epoch [20/20], Step [196/752], Loss: 0.6149
Epoch [20/20], Step [197/752], Loss: 1.3005
Epoch [20/20], Step [198/752], Loss: 1.0072
Epoch [20/20], Step [199/752], Loss: 0.6902
Epoch [20/20], Step [200/752], Loss: 0.8725
Epoch [20/20], Step [201/752], Loss: 0.7352
Epoch [20/20], Step [202/752], Loss: 0.6529
Epoch [20/20], Step [203/752], Loss: 1.1203
Epoch [20/20], Step [204/752], Loss: 0.6377
Epoch [20/20], Step [205/752], Loss: 0.6395
Epoch [20/20], Step [206/752], Loss: 1.2576
Epoch [20/20], Step [207/752], Loss: 0.6742
Epoch [20/20], Step [208/752], Loss: 1.0627
Epoch [20/20], Step [209/752], Loss: 1.6031
Epoch [20/20], Step [210/752], Loss: 1.0647
Epoch [20/20], Step [211/752], Loss: 0.9612
Epoch [20/20], Step [212/752], Loss: 0.6719
Epoch [20/20], Step [213/752], Loss: 1.4273
Epoch [20/20], Step [214/752], L

Epoch [20/20], Step [381/752], Loss: 0.8969
Epoch [20/20], Step [382/752], Loss: 0.7301
Epoch [20/20], Step [383/752], Loss: 0.9509
Epoch [20/20], Step [384/752], Loss: 0.9387
Epoch [20/20], Step [385/752], Loss: 0.4886
Epoch [20/20], Step [386/752], Loss: 0.4377
Epoch [20/20], Step [387/752], Loss: 1.1144
Epoch [20/20], Step [388/752], Loss: 0.5421
Epoch [20/20], Step [389/752], Loss: 2.3878
Epoch [20/20], Step [390/752], Loss: 0.5148
Epoch [20/20], Step [391/752], Loss: 0.4448
Epoch [20/20], Step [392/752], Loss: 0.5447
Epoch [20/20], Step [393/752], Loss: 1.1550
Epoch [20/20], Step [394/752], Loss: 0.8710
Epoch [20/20], Step [395/752], Loss: 0.8802
Epoch [20/20], Step [396/752], Loss: 0.4724
Epoch [20/20], Step [397/752], Loss: 1.2531
Epoch [20/20], Step [398/752], Loss: 1.2116
Epoch [20/20], Step [399/752], Loss: 0.4368
Epoch [20/20], Step [400/752], Loss: 1.4911
Epoch [20/20], Step [401/752], Loss: 0.3150
Epoch [20/20], Step [402/752], Loss: 0.4470
Epoch [20/20], Step [403/752], L

Epoch [20/20], Step [570/752], Loss: 1.5220
Epoch [20/20], Step [571/752], Loss: 1.1059
Epoch [20/20], Step [572/752], Loss: 0.7682
Epoch [20/20], Step [573/752], Loss: 1.0792
Epoch [20/20], Step [574/752], Loss: 0.5300
Epoch [20/20], Step [575/752], Loss: 0.6468
Epoch [20/20], Step [576/752], Loss: 1.4279
Epoch [20/20], Step [577/752], Loss: 0.3818
Epoch [20/20], Step [578/752], Loss: 0.3077
Epoch [20/20], Step [579/752], Loss: 0.8026
Epoch [20/20], Step [580/752], Loss: 0.8776
Epoch [20/20], Step [581/752], Loss: 0.6320
Epoch [20/20], Step [582/752], Loss: 0.4012
Epoch [20/20], Step [583/752], Loss: 0.5782
Epoch [20/20], Step [584/752], Loss: 0.6574
Epoch [20/20], Step [585/752], Loss: 0.4841
Epoch [20/20], Step [586/752], Loss: 0.9932
Epoch [20/20], Step [587/752], Loss: 0.7725
Epoch [20/20], Step [588/752], Loss: 1.0837
Epoch [20/20], Step [589/752], Loss: 0.7557
Epoch [20/20], Step [590/752], Loss: 0.6595
Epoch [20/20], Step [591/752], Loss: 0.9979
Epoch [20/20], Step [592/752], L

In [48]:
model.eval()
# eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
total_preds = np.array([])
total_labels = np.array([])
with torch.no_grad():
    correct = 0
    total = 0
    for i in test_loader:
        images = i['outer_product']
        labels = i['Label']
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images) 
        outputs = outputs.cpu().detach().numpy().flatten()
        labels =labels.cpu().detach().numpy().flatten()
        total_preds = np.concatenate([total_preds, outputs])
        total_labels = np.concatenate([total_labels, labels])
#         total_preds = torch.cat(total_preds, outputs.cpu(), 0 )
#         total_labels = torch.cat(total_labels, labels.cpu(), 0)
#         break

In [60]:
def mse(y,f):
    mse = ((y - f)**2).mean(axis=0)
    return mse
def pearson(y,f):
    rp = np.corrcoef(y, f)[0,1]
    return rp

In [59]:
mse(total_preds, total_labels)

0.8059452367125904

In [61]:
pearson(total_preds, total_labels)

0.28954438655076936